# Day 15: Oxygen System

In [143]:
from collections import defaultdict
from queue import Queue

In [4]:
class Instruction:
    def __init__(self, opcode, length, param_locations, param_values, memory):
        self.opcode = opcode
        self.length = length
        self.param_locations = param_locations
        self.param_values = param_values
        self.memory = memory
        
    def get_val(self, i):
        return self.param_values[i]
    
    def set_val(self, i, val):
        self.memory[self.param_locations[i]] = val

In [5]:
class Add(Instruction):
    def call(self):
        self.set_val(2, self.get_val(0) + self.get_val(1))

In [6]:
class Multiply(Instruction):
    def call(self):
        self.set_val(2, self.get_val(0) * self.get_val(1))

In [7]:
class Input(Instruction):
    def call(self, input_value):
        self.set_val(0, input_value)

In [8]:
class Output(Instruction):
    def call(self):
        return self.get_val(0)

In [9]:
class JumpIfTrue(Instruction):
    def call(self):
        return self.get_val(1) if self.get_val(0) else None

In [10]:
class JumpIfFalse(Instruction):
    def call(self):
        return self.get_val(1) if not self.get_val(0) else None

In [11]:
class LessThan(Instruction):
    def call(self):
        val = 1 if self.get_val(0) < self.get_val(1) else 0
        self.set_val(2, val)

In [12]:
class Equals(Instruction):
    def call(self):
        val = 1 if self.get_val(0) == self.get_val(1) else 0
        self.set_val(2, val)

In [13]:
class RelativeBaseOffset(Instruction):
    def call(self):
        return self.get_val(0)

In [14]:
class Halt(Instruction):
    pass

In [15]:
class IntcodeComputer:
    # Instruction opcodes
    ADD = 1
    MULTIPLY = 2
    INPUT = 3
    OUTPUT = 4
    JUMP_IF_TRUE = 5
    JUMP_IF_FALSE = 6
    LESS_THAN = 7
    EQUALS = 8
    RELATIVE_BASE_OFFSET = 9
    HALT = 99
    
    LENGTHS = {
        ADD: 4,
        MULTIPLY: 4,
        INPUT: 2,
        OUTPUT: 2,
        JUMP_IF_TRUE: 3,
        JUMP_IF_FALSE: 3,
        LESS_THAN: 4,
        EQUALS: 4,
        RELATIVE_BASE_OFFSET: 2,
        HALT: 1,
    }
    
    INSTRUCTIONS = {
        ADD: Add,
        MULTIPLY: Multiply,
        INPUT: Input,
        OUTPUT: Output,
        JUMP_IF_TRUE: JumpIfTrue,
        JUMP_IF_FALSE: JumpIfFalse,
        LESS_THAN: LessThan,
        EQUALS: Equals,
        RELATIVE_BASE_OFFSET: RelativeBaseOffset,
        HALT: Halt,
    }
    
    POSITION_MODE = '0'
    IMMEDIATE_MODE = '1'
    RELATIVE_MODE = '2'
    
    def __init__(self, program):
        self.memory = program.copy()
        self.memory_pointer = 0
        self.relative_base = 0
        self.inputs = []
        
    def create_instruction(self):
        op = str(self.memory[self.memory_pointer])
        opcode = int(op[-2:])    
        length = self.LENGTHS[opcode]
        args = [self.memory[self.memory_pointer + i] for i in range(1, length)]
        modes = op[:-2].zfill(length - 1)[::-1]
        locs = []
        vals = []
        for arg, mode in zip(args, modes):
            if mode == self.POSITION_MODE: 
                locs.append(arg)
                vals.append(self.memory[arg])
            elif mode == self.IMMEDIATE_MODE:
                locs.append(None)
                vals.append(arg)
            elif mode == self.RELATIVE_MODE:
                locs.append(arg + self.relative_base)
                vals.append(self.memory[arg + self.relative_base])

        return self.INSTRUCTIONS[opcode](opcode, length, locs, vals, self.memory)
    
    def queue_input(self, input_value):
        self.inputs.append(input_value)
        
    def run(self):
        while True:
            instruction = self.create_instruction()
            if instruction.opcode == self.HALT:
                break
            elif instruction.opcode == self.INPUT:
                value = instruction.call(self.inputs.pop(0))
            else:
                value = instruction.call()
            
            self.memory_pointer += instruction.length
            if instruction.opcode in [self.JUMP_IF_TRUE, self.JUMP_IF_FALSE] and value is not None:
                self.memory_pointer = value
            elif instruction.opcode == self.RELATIVE_BASE_OFFSET:
                self.relative_base += value
            elif instruction.opcode == self.OUTPUT:
                yield value

    @staticmethod
    def to_program(data):
        return defaultdict(int, enumerate(data.copy()))

In [16]:
with open('day_15.txt') as f:
    data = [int(token) for token in f.readlines()[0].strip().split(',')]

## Part 1
For this part we are first going to traverse the entire maze to create the adjacency list of all positions in the maze. 

Then we're going to use a breadth-first search approach to get all distances between the start position and the oxygen.

In [164]:
NORTH = 1
SOUTH = 2
WEST = 3
EAST = 4

HIT_WALL = 0
MADE_STEP = 1
FOUND_OXYGEN = 2

START_LOC = 0j

STEP_DIR = {
    NORTH: 1j,
    SOUTH: -1j,
    WEST: -1 + 0j,
    EAST: 1 + 0j
}

WALL_CRUMB = -1

START = "🎅🏻"
WALL = "🌲"
PATH = "🌫"
OXYGEN = '🎁'

In order to exhaustively traverse the maze with the robot, I wanted to make sure not to accidentally have a traversal scheme that would miss any pathways or get stuck in any loops. 

To achieve this, I leave a trail of breadcrumbs (a dict called `crumbs` from position to an integer) everywhere the robot goes. When the robot needs to choose which direction to move, it will always choose the direction with the least breadcrumbs, i.e. the path it has visited the least. 

This ensures the robot won't get stuck in some kind of weird loop based on hardcoded navigation rules.

I also wanted to be able to know when the robot has explored all pathways in the maze for a halting condition. To do this, I keep a set called `unvisited`. When the robot is at one location, any unvisited adjacent locations get added to `unvisited`. When the robot steps into a new position, we remove it from `unvisited`. The function halts when `unvisited` is empty.

In [138]:
def build_maze(computer, start_loc):
    edges = defaultdict(set)
    crumbs = defaultdict(int)
    loc = start_loc
    oxygen_loc = None
    crumbs[loc] = 1
    direction = NORTH
    computer.queue_input(direction)
    unvisited = set()
    for status in computer.run():
        tried_loc = loc + STEP_DIR[direction]
        unvisited.discard(tried_loc)
        
        if status == HIT_WALL:
            crumbs[tried_loc] = WALL_CRUMB
        else:
            edges[loc].add(tried_loc)
            edges[tried_loc].add(loc)
            loc = tried_loc
            crumbs[loc] += 1
        
        if status == FOUND_OXYGEN:
            oxygen_loc = loc
        
        min_crumbs = None
        for d in [NORTH, SOUTH, WEST, EAST]:
            neighbor = loc + STEP_DIR[d]
            if crumbs[neighbor] == WALL_CRUMB:
                continue
                
            crumb_count = crumbs[neighbor]
            if crumb_count == 0:
                unvisited.add(neighbor)
            if min_crumbs is None or crumb_count < min_crumbs:
                min_crumbs = crumb_count
                direction = d
        if not unvisited:
            break
        computer.queue_input(direction)
    return [edges, oxygen_loc]

In [139]:
program = IntcodeComputer.to_program(data)
computer = IntcodeComputer(program)

In [140]:
edges, oxygen_loc = build_maze(computer, START_LOC)

Once we have the maze in adjacency list form (`edges`), we can use `get_distances` to find the minimum distance from the starting point to the oxygen location.

In [154]:
def get_distances(edges, start_loc):
    distances = {start_loc: 0}
    loc_queue = Queue()
    loc_queue.put(start_loc)
    while not loc_queue.empty():
        loc = loc_queue.get()
        for neighbor in edges[loc]:
            if neighbor not in distances or distances[loc] + 1 < distances[neighbor]:
                distances[neighbor] = distances[loc] + 1
                loc_queue.put(neighbor)
    return distances

In [155]:
get_distances(edges, START_LOC)[oxygen_loc]

232

## Part 2
We've already done all the hard work! For Part 2, we just need to find the max distance from the oxygen to any other point in the maze.

In [156]:
max(get_distances(edges, oxygen_loc).values())

320

Just for fun, I also wrote a function to print the maze.

In [165]:
def print_maze(path_locs, start_loc, oxygen_loc):
    maze = defaultdict(lambda: WALL)
    for loc in path_locs:
        maze[loc] = PATH
    maze[start_loc] = START
    maze[oxygen_loc]= OXYGEN
    min_x = min([int(loc.real) for loc in path_locs]) - 1
    max_x = max([int(loc.real) for loc in path_locs]) + 1
    min_y = min([int(loc.imag) for loc in path_locs]) - 1
    max_y = max([int(loc.imag) for loc in path_locs]) + 1
    for y in range(max_y, min_y - 1, -1):
        row = [maze[complex(x, y)] for x in range(min_x, max_x + 1)]
        print("".join(row))

In [166]:
print_maze(edges, START_LOC, oxygen_loc)

🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲
🌲🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌲🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲
🌲🌫🌲🌫🌲🌫🌲🌲🌲🌲🌲🌲🌲🌫🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌫🌲🌲🌲🌫🌲🌫🌲
🌲🌫🌲🌫🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌲🌫🌲
🌲🌫🌲🌫🌲🌫🌲🌫🌲🌲🌲🌲🌲🌲🌲🌫🌲🌫🌲🌲🌲🌲🌲🌫🌲🌫🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌲🌲🌫🌲
🌲🌫🌲🌫🌲🌫🌲🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲🌫🌲🌫🌫🌫🌲🌫🌲🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲
🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌫🌲🌫🌲🌲🌲🌫🌲🌫🌲🌲🌲🌫🌲🌲🌲🌫🌲🌲🌲🌫🌲🌫🌲🌲🌲🌲🌲🌫🌲
🌲🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲🌫🌲🌫🌲🌫🌫🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲🌫🌲🎁🌲🌫🌫🌫🌲
🌲🌫🌲🌲🌲🌫🌲🌲🌲🌲🌲🌲🌲🌫🌲🌫🌲🌲🌲🌲🌲🌲🌲🌲🌲🌫🌲🌲🌲🌲🌲🌲🌲🌫🌲🌫🌲🌫🌲🌲🌲
🌲🌫🌲🌫🌫🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲
🌲🌫🌲🌫🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌫🌲🌫🌲🌫🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲
🌲🌫🌲🌫🌲🌫🌫🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌲🌫🌲🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲
🌲🌫🌲🌫🌲🌫🌲🌲🌲🌫🌲🌫🌲🌲🌲🌫🌲🌫🌲🌫🌲🌫🌲🌲🌲🌲🌲🌫🌲🌲🌲🌫🌲🌫🌲🌫🌲🌫🌲🌫🌲
🌲🌫🌫🌫🌲🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌲🌫🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌫🌫🌲🌫🌲🌫🌲🌫🌲🌫🌲🌫🌲
🌲🌫🌲🌲🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌫🌲🌫🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌫🌲🌫🌲🌫🌲🌫🌲🌫🌲
🌲🌫🌫🌫🌫🌫🌫🌫🌲🌫🌫🌫🌲🌫🌲🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌲🌫🌫🌫🌲🌫🌲
🌲🌲🌲🌲🌲🌲🌲🌲🌲🌫🌲🌲🌲🌫🌲🌫🌲🌲🌲🌫🌲🌫🌲🌫🌲🌫🌲🌫🌲🌫🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲
🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌲🌫🌲🌫🌫🌫🌫🌫🌲🌫🌫🌫🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌫🌫🌲🌫🌲
🌲🌫🌲🌫🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌫🌲🌫🌲🌲🌲🌲🌲🌲🌲🌲🌲🌫🌲🌲🌲🌫🌲🌲🌲🌲🌲🌲🌲🌫🌲
🌲🌫🌫🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲🌫🌲🌫🌲🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲
🌲🌫🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌫🌲🌫🌲🌲🌲🌫🌲🌫🌲🌫🌲🌫🌲🌲🌲🌫🌲🌫🌲🌫🌲🌲🌲🌲🌲🌲🌲
🌲🌫🌫🌫🌲🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲🎅🏻🌲🌫🌲🌫🌫🌫🌲🌫🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲
🌲🌲🌲🌫🌲🌫🌲🌫🌲🌲🌲🌲🌲🌫🌲🌲🌲🌲🌲🌲🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌲🌲🌲🌲🌫🌲🌲🌲🌫🌲
🌲🌫🌫🌫🌲🌫🌲🌫🌫🌫🌲🌫🌲🌫🌲🌫🌫🌫🌫🌫🌫🌫🌲🌫🌫🌫🌫🌫🌲🌫🌫🌫🌫